**1)** Qual a média de valor total (total_amount) recebido em um mês considerando todos os yellow táxis da frota?

In [0]:
%sql
SELECT
    trip_year  AS ano,
    trip_month AS mes,
    COUNT(trip_month) as qty,
    ROUND(AVG(total_amount), 2) AS media_total_amount
FROM   ifood_datalake.silver.yellow_tripdata_silver
GROUP  BY trip_year, trip_month
ORDER  BY trip_year, trip_month;


**2)** Qual a média de passageiros (passenger_count) por cada hora do dia que pegaram táxi no mês de maio considerando todos os táxis da frota?

In [0]:
%sql
WITH union_df AS (
    SELECT passenger_count,
           COALESCE(tpep_pickup_datetime, tpep_pickup_datetime) AS pickup_ts
    FROM   ifood_datalake.silver.yellow_tripdata_silver
    UNION ALL
    SELECT passenger_count,
           COALESCE(tpep_pickup_datetime,  tpep_pickup_datetime) AS pickup_ts
    FROM   ifood_datalake.silver.green_tripdata_silver
)
SELECT
    HOUR(pickup_ts)                  AS pickup_hour,
    ROUND(AVG(passenger_count), 2)   AS media_passageiros
FROM   union_df
GROUP  BY pickup_hour
ORDER  BY pickup_hour;


Outras análises:

In [0]:
%sql
WITH ref_dates AS (
  SELECT date_id
  FROM   ifood_datalake.gold.dim_date
)
SELECT   
    f.taxi_type,
    d.year   AS ano,
    d.month  AS mes,
    SUM(f.num_trips)                   AS total_viagens,
    SUM(f.total_passengers)           AS total_passageiros,
    ROUND(SUM(f.total_revenue), 2)    AS receita_total,
    ROUND(SUM(f.distance_mi), 2)      AS milhas_totais,
    ROUND(SUM(f.total_duration), 2)   AS duracao_total_min,
    ROUND(SUM(f.total_revenue) / SUM(f.num_trips), 2)           AS ticket_medio,
    ROUND(SUM(f.total_revenue) / SUM(f.total_passengers), 2)    AS receita_por_passageiro,
    ROUND(SUM(f.distance_mi) / SUM(f.num_trips), 2)             AS distancia_media_mi,
    ROUND(SUM(f.total_duration) / SUM(f.num_trips), 2)          AS duracao_media_min,
    ROUND(SUM(f.total_passengers) / SUM(f.num_trips), 2)        AS ocupacao_media,
    ROUND(SUM(f.total_revenue) / NULLIF(SUM(f.distance_mi), 0), 2) AS receita_por_milha,
    ROUND(SUM(f.total_duration) / NULLIF(SUM(f.distance_mi), 0), 2) AS tempo_por_milha,
    ROUND(AVG(f.load_factor), 2)                                AS fator_de_carga_medio

FROM     ifood_datalake.gold.fact_trips f
JOIN     ifood_datalake.gold.dim_date d
              ON f.pickup_date_id = d.date_id
JOIN     ref_dates r           
              ON r.date_id = d.date_id
GROUP BY f.taxi_type, d.year, d.month
ORDER BY d.year, d.month, f.taxi_type;

In [0]:
%sql
SELECT   
    f.taxi_type,
    SUM(f.num_trips)                   AS total_viagens,
    SUM(f.total_passengers)           AS total_passageiros,
    ROUND(SUM(f.total_revenue), 2)    AS receita_total,
    ROUND(SUM(f.distance_mi), 2)      AS milhas_totais,
    ROUND(SUM(f.total_duration), 2)   AS duracao_total_min,
    
    ROUND(SUM(f.total_revenue) / SUM(f.num_trips), 2)           AS ticket_medio,
    ROUND(SUM(f.total_revenue) / SUM(f.total_passengers), 2)    AS receita_por_passageiro,
    ROUND(SUM(f.distance_mi) / SUM(f.num_trips), 2)             AS distancia_media_mi,
    ROUND(SUM(f.total_duration) / SUM(f.num_trips), 2)          AS duracao_media_min,
    ROUND(SUM(f.total_passengers) / SUM(f.num_trips), 2)        AS ocupacao_media,
    ROUND(SUM(f.total_revenue) / NULLIF(SUM(f.distance_mi), 0), 2) AS receita_por_milha,
    ROUND(SUM(f.total_duration) / NULLIF(SUM(f.distance_mi), 0), 2) AS tempo_por_milha,
    ROUND(AVG(f.load_factor), 2)                                AS fator_de_carga_medio

FROM ifood_datalake.gold.fact_trips f
GROUP BY f.taxi_type
ORDER BY receita_total DESC;
